In [1]:
#####
# 1.最新四半期決算の営業利益率は低い
# 2.最新四半期決算の売上高成長率が10%以上
# 3.最新四半期決算の対売上高営業利益成長率が20%以上
# 4.次期決算を、以下の方法で予測した場合において、営業利益率が前年同期比2倍以上になる
# --> 最新四半期決算の売上高成長率と対売上高営業利益成長率を、次期前年四半期決算に適用して次期決算を予想する

In [2]:
# 読み込みファイルパスの設定とimportしたいmoduleパス(pythonパス)の設定
from pathlib import Path
import os, sys

CURRENT_DIR = Path(os.getcwd())
PJ_DIR = CURRENT_DIR.parent.parent
LIB_DIR = PJ_DIR / "lib"
DATA_DIR = PJ_DIR / "data" 

sys.path.append(str(LIB_DIR))

# notebook内で利用するmoduleのimport
from lib_dataprocess import KessanPl, PricelistFig, PricelistPl, MeigaralistPl
from lib_dataprocess import ShikihoPl, FinancequotePl, print_finance_quote, PricelistFig, KessanFig, IndexPricelistPl
import polars as pl
from datetime import date
from dateutil.relativedelta import relativedelta

RevPl = PricelistPl("reviced_pricelist.parquet")
RawPL = PricelistPl("raw_pricelist.parquet")
MPL = MeigaralistPl()
SPL = ShikihoPl()

In [3]:
#####
##### valuation_date時点の発表済最新四半期決算の業績成長率一覧(growth_rate_df)の作成
#####

In [4]:
valuation_date = date(2024, 8, 1)

KPL1 = KessanPl()
KPL2 = KessanPl()
KPL1.with_columns_growthrate_lastyear()
KPL2.with_columns_diff_growthrate()
KPL1.df = KPL1.df.filter(pl.col("settlement_type")=="四")
KPL2.df = KPL2.df.filter(pl.col("settlement_type")=="四")
df = KPL1.df.join(KPL2.df, on=["code", "settlement_date"], how="left")
KPL3 = KessanPl(df)
df =KPL3.get_latest_quater_settlements(valuation_date)
df = df.select([
    "code", "settlement_date", "announcement_date", "sales", "operating_income", "sales_growthrate", "diff_operating_income_growthrate"
])
df = df.rename({"settlement_date": "評価日発表済最新四半期決算"})
growth_rate_df = df
growth_rate_df

code,評価日発表済最新四半期決算,announcement_date,sales,operating_income,sales_growthrate,diff_operating_income_growthrate
i64,date,date,i64,i64,f64,f64
1301,2024-03-31,2024-05-10,59990,1857,2.1,155.0
1332,2024-03-31,2024-05-14,205894,3308,8.5,13.6
1333,2024-03-31,2024-05-07,243328,1176,1.6,-37.3
1375,2024-03-31,2024-05-09,11807,-803,21.1,-5.0
1376,2024-05-31,2024-07-10,23224,1150,1.9,-16.0
…,…,…,…,…,…,…
9993,2024-05-31,2024-07-11,24969,-408,1.8,-108.5
9994,2024-03-31,2024-05-15,36543,1082,3.1,63.8
9995,2024-03-31,2024-05-07,18708,63,7.7,0.1
